In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Import des librairies 

#Numpy & pandas
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

# image and CV
import cv2 as cv

# Fichiers
import pickle

# Plot
from matplotlib import pyplot as plt
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# IpyWidgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import AppLayout, Layout

# Outil de visualisation de LDA
import pyLDAvis
import pyLDAvis.sklearn

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = 'D:/Data/Openclassroom/P6/Data'
file_reviews_samples = PATH + '/review_samples.json'

# Test de l'API

In [203]:
# API
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from config import API_KEY_YELP

# IpyWidgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import AppLayout, Layout

# Define the header
header = {'Authorization':f"bearer {API_KEY_YELP}",
         'Content-Type': 'application/json'}

# Build the request framework
transport = RequestsHTTPTransport(url = 'https://api.yelp.com/v3/graphql', headers = header, use_json = True)

# Create the client
client = Client(transport = transport, fetch_schema_from_transport = True)

def query(n_target, rating_target, location):
    # Init
    reviews = []
    n_loops = 0
    
    while (len(reviews) < n_target) and (n_loops < 10): # second condition to cap API ressources usage

        offset = n_loops*50
        n_loops = n_loops + 1

        # Query reviews from Restaurants in san francisco
        offset_text = "{offset: .0f}".format(offset=offset)
        query_text = '{search(categories: "restaurant",\
                location: ' + location + ',\
                offset: ' + offset_text + ',\
                limit: 50)\
                { business\
                     {  name\
                        reviews\
                                 { rating\
                                   text\
                                 }\
                    }\
                }\
            }'
        query = gql(query_text)
        resp = client.execute(query)

        # Extract reviews with rating_target stars
        l_biz = resp['search']['business']
        for i in range(len(l_biz)):
            l_revs = l_biz[i]['reviews']
            for j in range(len(l_revs)):
                if l_revs[j]['rating'] == rating_target:
                    reviews.append(l_revs[j]['text'])

        print(f"Passage {n_loops}, nombre de revues {rating_target} étoile(s) = {len(reviews)}")
    
    w_api3.value = "\n".join(reviews)

    
w_api0 = widgets.Dropdown(
    options=[('"new york"'), ('"san francisco"'), ('"miami"')],
    value='"new york"',
    description='Ville:',
)

w_api1 = widgets.IntText(
    value= 5,
    description='Etoiles:',
    disabled=False
)

w_api2 = widgets.IntText(
    value= 5,
    description='# revues:',
    disabled=False
)

w_api3 = widgets.Textarea(
    value=' ',
    placeholder='Update',
    description='     ',
    disabled=False,
    layout={'height': '370px', 'width': '500px'}
)

out_api0 = widgets.interactive_output(query, {'n_target': w_api2,
                                              'rating_target': w_api1,
                                              'location': w_api0
                                             })

widgets.VBox([widgets.VBox([w_api0, w_api1, w_api2]),
              widgets.Box([w_api3, out_api0],
                           layout={'height': '400px', 'width': 'auto'}
                          ),
             ],
            )

# Détection des sujets de mécontentement

In [4]:
%matplotlib inline
vis = pickle.load(open('vis.p', "rb"))
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [5]:
def filter_docs(topic, seuil, df_docs):
    idx = df_docs[topic] > seuil
    df = df_docs.loc[idx]
    return(df)

def sample_index(topic, seuil, df_docs):
    df = filter_docs(topic, seuil, df_docs)
    idx = np.random.randint(0, len(df))
    return(df.index[idx])

def display_text(idx, df_docs):
    sample = df_docs.loc[idx]
    w06.value = sample['text']
    
def display_graph(idx, df_docs):
    sample = df_docs.loc[idx]
    x = ['topic1', 'topic2', 'topic3', 'topic4']
    y = []
    for t in x:
        y.append(sample[t])
    fig = px.bar(y=y, x=x, width=400, height=400, 
                 labels={'x':'Proportion des topics dans le document', 'y':''},
                )
    fig.show('notebook')
    
def display_div(idx, df_docs):
    fig = px.histogram(df_docs, x='div', width=800, height=400,
                       title='Histogramme de la diversité des documents',
                       labels={'div':'Mesure de la diversité', 'count':' '},
                       nbins=500
                )
    fig.show('notebook')
lda = pickle.load(open('lda.p', "rb"))
vocabulary_ranked = lda['vocabulary_ranked']
df_docs = lda['df_docs']

w01 = widgets.Dropdown(
    options=[('topic1'), ('topic2'), ('topic3'), ('topic4')],
    value='topic1',
    description='Topic:',
)

w02 = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='Seuil:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

w04 = widgets.IntText(
    value=0,
    description='Index:',
    disabled=False
)

w05 = widgets.Button(
    description='Update',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Update',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)


w06 = widgets.Textarea(
    value=' ',
    placeholder='Update',
    description='     ',
    disabled=False,
    layout={'height': '370px', 'width': '500px'}
)

w07 = widgets.IntText(
    value=0,
    description='Index:',
    disabled=False
)

def on_click_w05(*args):
    w04.value = sample_index(w01.value, w02.value, df_docs)

w05.on_click(on_click_w05)

out01 = widgets.interactive_output(display_text, {'idx': w04, 'df_docs': fixed(df_docs)})
out02 = widgets.interactive_output(display_graph, {'idx': w04, 'df_docs': fixed(df_docs)})
out03 = widgets.interactive_output(display_div, {'idx' : w07, 'df_docs': fixed(df_docs)})


widgets.VBox([widgets.HBox([w01, w02, w05]),
              widgets.HBox([w06, out02],
                           layout={'height': '400px', 'width': 'auto'}
                          ),
              out03
             ],
            )

# Exemples de pré-traitement d'images

In [ ]:
def hist(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256],density=True)
    cdf = hist.cumsum()
    return(hist, cdf)

def mag_dft(I):
    if I is None:
        print('Error opening image')
        return -1
    
    rows, cols = I.shape
    m = cv.getOptimalDFTSize( rows )
    n = cv.getOptimalDFTSize( cols )
    padded = cv.copyMakeBorder(I, 0, m - rows, 0, n - cols, cv.BORDER_CONSTANT, value=[0, 0, 0])
    
    planes = [np.float32(padded), np.zeros(padded.shape, np.float32)]
    complexI = cv.merge(planes)         # Add to the expanded another plane with zeros
    
    cv.dft(complexI, complexI)         # this way the result may fit in the source matrix
    
    cv.split(complexI, planes)                   # planes[0] = Re(DFT(I), planes[1] = Im(DFT(I))
    cv.magnitude(planes[0], planes[1], planes[0])# planes[0] = magnitude
    magI = planes[0]
    
    matOfOnes = np.ones(magI.shape, dtype=magI.dtype)
    cv.add(matOfOnes, magI, magI) #  switch to logarithmic scale
    cv.log(magI, magI)
    
    magI_rows, magI_cols = magI.shape
    # crop the spectrum, if it has an odd number of rows or columns
    magI = magI[0:(magI_rows & -2), 0:(magI_cols & -2)]
    cx = int(magI_rows/2)
    cy = int(magI_cols/2)
    q0 = magI[0:cx, 0:cy]         # Top-Left - Create a ROI per quadrant
    q1 = magI[cx:cx+cx, 0:cy]     # Top-Right
    q2 = magI[0:cx, cy:cy+cy]     # Bottom-Left
    q3 = magI[cx:cx+cx, cy:cy+cy] # Bottom-Right
    tmp = np.copy(q0)               # swap quadrants (Top-Left with Bottom-Right)
    magI[0:cx, 0:cy] = q3
    magI[cx:cx + cx, cy:cy + cy] = tmp
    tmp = np.copy(q1)               # swap quadrant (Top-Right with Bottom-Left)
    magI[cx:cx + cx, 0:cy] = q2
    magI[0:cx, cy:cy + cy] = tmp
    
    cv.normalize(magI, magI, 0, 1, cv.NORM_MINMAX) # Transform the matrix with float values into a
    return(magI)

def process_image(processing): #Bruit, Exposition, Rotation

    if processing == 'Bruit':
        src = cv.imread(PATH+"/simba_bruite3.png", cv.IMREAD_GRAYSCALE)
        hist_src, cdf_src = hist(src)
        dft_src = mag_dft(src)

        blur_coef = 7
        dst = cv.blur(src,(blur_coef,blur_coef))
        hist_dst, cdf_dst = hist(dst)
        dft_dst = mag_dft(dst)

    if processing == 'Exposition':

        src = cv.imread(PATH+"/simba_sous_expo.png", cv.IMREAD_GRAYSCALE)
        hist_src, cdf_src = hist(src)
        dft_src = mag_dft(src)

        dst = cv.equalizeHist(src)
        hist_dst, cdf_dst = hist(dst)
        dft_dst = mag_dft(dst)


    if processing == 'Rotation':

        src = cv.imread(PATH+"/simba.png", cv.IMREAD_GRAYSCALE)      
        
        rows,cols = src.shape
        M = cv.getRotationMatrix2D((cols/2,rows/2),45,1.4)
        dst = cv.warpAffine(src,M,(cols,rows))

        hist_src, cdf_src = hist(src)
        dft_src = mag_dft(src)

        hist_dst, cdf_dst = hist(dst)
        dft_dst = mag_dft(dst)
        
    return((src, dst, hist_src, hist_dst, cdf_src, cdf_dst, dft_src, dft_dst))

def plot(src, dst, hist_src, hist_dst, cdf_src, cdf_dst, dft_src, dft_dst):
    
    fig, ax = plt.subplots(3,2, figsize=(10,10))

    # Images originale et transformée
    ax[0,0].set_title("Image originale")
    ax[0,0].imshow(src,cmap="gray")

    ax[0,1].set_title("Image transformée")
    ax[0,1].imshow(dst,cmap="gray")

    # Histogrammes de l'image originale
    ax[1,0].set_title("Histogramme des niveaux de gris original")
    ax[1,0].plot(hist_src, label='Densité')
    ax[1,0].legend(loc=2)

    ax10 = ax[1,0].twinx()
    ax10.plot(cdf_src, color='r', label='Cumul')
    ax10.legend(loc=9)

    # Histogrammes de l'image transformée
    ax[1,1].set_title("Histogramme des niveaux de gris transformé")
    ax[1,1].plot(hist_dst, label='Densité')
    ax[1,1].legend(loc=2)

    ax11 = ax[1,1].twinx()
    ax11.plot(cdf_dst, color='r', label='Cumul')
    ax11.legend(loc=9)

    # DFT des images originale et transformée
    ax[2,0].set_title("DFT originale")
    ax[2,0].imshow(dft_src, cmap="gray")

    ax[2,1].set_title("DFT transformée")
    ax[2,1].imshow(dft_dst, cmap="gray")

    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                        wspace=0.35)
    
def demo(processing): # Bruit, Exposition, Rotation
    
    d = process_image(processing)
    plot(*d)

In [ ]:
w1 = widgets.Dropdown(
    options=[('Bruit'), ('Exposition'), ('Rotation')],
    value='Bruit',
    description='A traiter:',
)

out1 = widgets.interactive_output(demo, {'processing': w1})

widgets.VBox([widgets.HBox([w1]), out1])

# Labellisation des photos par thème

In [ ]:
def plot_clusters(bow_tsne2, labels2, km_labels_):
    
    enc = OrdinalEncoder()

    labels = labels2.copy()
    labels2 = enc.fit_transform(labels2.reshape(-1,1))

    df_plot2 = pd.DataFrame(bow_tsne2, columns=['tsne_X', 'tsne_Y'])
    df_plot2['km_label'] = km_labels_
    df_plot2['label_enc'] = labels2

    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=("Données réelles", "Données clusterisées"))

    fig.add_trace(
        go.Scatter(x=df_plot2['tsne_X'],
                   y=df_plot2['tsne_Y'],
                   mode='markers',
                   marker=dict(color=df_plot2['label_enc']),
                   hovertext = list(labels),
                  ),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(x=df_plot2['tsne_X'],
                   y=df_plot2['tsne_Y'],
                   mode='markers',
                   marker=dict(color=df_plot2['km_label']),
                   hovertext = list(labels),
                  ),
        row=1, col=2
    )

    fig.update_layout(height=600, width=800, title_text="Projection en 2D - Clustering KM")

    fig.show()
    

def plot_results(algorithm, perplexity, tsne_results_sift, tsne_results_cnn):
    
    if algorithm == 'SIFT':
        tsne_results = tsne_results_sift
    if algorithm == 'CNN':
        tsne_results = tsne_results_cnn
        
    i=tsne_results['perplexity_list'].index(perplexity)
    ari = tsne_results['score_list'][i]
    n_inliers = len(tsne_results['labels2_list'][i]) 
    n_outliers = tsne_results['n_outliers_list'][i]
    
    print(f"ARI = {ari}\nNombre d'inliers = {n_inliers}\nNombre d'outliers = {n_outliers}")

    bow_tsne2 = tsne_results['bow_tsne2_list'][i]
    labels2 = tsne_results['labels2_list'][i]
    km_labels_ = tsne_results['km_labels_list'][i]

    plot_clusters(bow_tsne2, labels2, km_labels_)


In [ ]:
tsne_results_sift = pickle.load(open('tsne_results_sift.p', 'rb'))
tsne_results_cnn = pickle.load(open('tsne_results_cnn.p', 'rb'))

w21 = widgets.Dropdown(
    options=[('SIFT'), ('CNN')],
    value='SIFT',
    description='Algorithme:',
)

w22 = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=5,
    description='Perplexity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


out2 = widgets.interactive_output(plot_results, {'algorithm': w21,
                                                 'perplexity': w22,
                                                 'tsne_results_sift': fixed(tsne_results_sift),
                                                 'tsne_results_cnn': fixed(tsne_results_cnn),
                                                })

widgets.VBox([widgets.HBox([w21, w22]), out2])